## Normal Distribution and Error Function

The Cumulative Distribution Function (CDF) of normal distribution is given by
$$\phi(x) = \int_{-\infty}^x \frac{1}{\sqrt{2\pi} \sigma} \text{e}^{-\frac{1}{2}(\frac{t-\mu}{\sigma})^2} dt$$
with the Probability Density Function (PDF) being $f(x) = \frac{1}{\sqrt{2\pi} \sigma} \text{e}^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ and the derivative of $\phi(x)$ is $f(x)$, i.e., $\phi'(x) = f(x)$.

There is no simple formula to evaluate the normal CDF. Instead, there are extensive tables and computational algorithms. One common approach is based on a relationship with the ``error function``. The error function, symbolized by __erf(x)__, is defined by
$$\text{erf}(x) = \frac{1}{\sqrt{\pi}} \int_{-x}^x \text{e}^{-t^2}dt$$

Thus, the error function can be related to the CDF of normal distribution:
$$\phi(x) =\frac{1}{2}[1 + \text{erf}(\frac{x-\mu}{\sqrt{2}\sigma})]$$

The proof is

$$\frac{1}{2}[1 + \text{erf}(\frac{x-\mu}{\sqrt{2}\sigma})] = \int_{-\infty}^0 \frac{1}{\sqrt{2\pi} \sigma} \text{e}^{-\frac{1}{2}(\frac{t-\mu}{\sigma})^2} dt + \frac{1}{2}\frac{1}{\sqrt{\pi}}\int_{-\frac{x-\mu}{\sqrt{2}\sigma}}^{\frac{x-\mu}{\sqrt{2}\sigma}} \text{e}^{-t^2}dt$$

The last integral can be simplified by

$$\int_{-\frac{x-\mu}{\sqrt{2}\sigma}}^{\frac{x-\mu}{\sqrt{2}\sigma}} \text{e}^{-t^2}dt = \int_{-\frac{x-\mu}{\sqrt{2}\sigma}}^0 \text{e}^{-t^2}dt + \int_{0}^{\frac{x-\mu}{\sqrt{2}\sigma}} \text{e}^{-t^2}dt = 2\int_{0}^{\frac{x-\mu}{\sqrt{2}\sigma}} \text{e}^{-t^2}dt (\text{ symmetric w.r.t. t})$$

Let $s = \sqrt{2}\sigma t + \mu$, then $0 \leq s \leq x$ and $dt = \frac{1}{\sqrt{2}\sigma} ds$,

$$\int_{0}^{\frac{x-\mu}{\sqrt{2}\sigma}} \text{e}^{-t^2}dt = \frac{1}{\sqrt{2}\sigma}\int_0^x \text{e}^{-\frac{1}{2}(\frac{s-\mu}{\sigma})^2}ds$$

Therefore,
$$\frac{1}{2}[1 + \text{erf}(\frac{x-\mu}{\sqrt{2}\sigma})] = \int_{-\infty}^0 \frac{1}{\sqrt{2\pi} \sigma} \text{e}^{-\frac{1}{2}(\frac{s-\mu}{\sigma})^2} ds + \frac{1}{\sqrt{2\pi}\sigma}\int_0^x \text{e}^{-\frac{1}{2}(\frac{s-\mu}{\sigma})^2}ds = \int_{-\infty}^x \frac{1}{\sqrt{2\pi} \sigma} \text{e}^{-\frac{1}{2}(\frac{s-\mu}{\sigma})^2} ds$$

Specifically, for truncated normal distribution,
$$\int_a^b z \cdot f(x)dx = 1, \text{ where } z \text{ is a scaler and is independent of } x$$

So,
$$z = \frac{1}{\int_a^b f(x)dx} = \frac{1}{\phi(b) - \phi(a)}$$

Denote $\Phi(x)$ as the CDF of ``standard normal distribution``, then 
$$\int_a^b f(x)dx = \int_a^b  \frac{1}{\sqrt{2\pi} \sigma} \text{e}^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2} dx = \int_{\frac{a-\mu}{\sigma}}^{\frac{b-\mu}{\sigma}} \frac{1}{\sqrt{2\pi}}\text{e}^{-\frac{1}{2} t^2} dt = \Phi(\frac{b-\mu}{\sigma})- \Phi(\frac{a-\mu}{\sigma})$$

Then the probability density function (PDF) of truncated normal distribution for $a \leq x \leq b$ is 
$$g(x) = z \cdot f(x) = \frac{1}{\Phi(\frac{b-\mu}{\sigma})- \Phi(\frac{a-\mu}{\sigma})} \cdot \frac{1}{\sqrt{2\pi} \sigma} \text{e}^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$$

If $a=0$ and $b = +\infty$, then $\Phi(\frac{b-\mu}{\sigma}) = 1$ and $\Phi(\frac{a-\mu}{\sigma}) = \Phi(-\frac{\mu}{\sigma})$, then the PDF of a normal distribution $N(x \mid \mu, \sigma^2)$ has a $\mu$ and $\sigma$-dependent scaling factor, i.e.,

$$g(x\mid \mu, \sigma^2) = \frac{1}{1 - \Phi(-\frac{\mu}{\sigma})} \frac{1}{\sqrt{2\pi} \sigma} \exp\left[{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2}\right]$$
$$\Phi(\omega) = \frac{1}{2}[1 + \text{erf}(\frac{\omega}{\sqrt{2}})] $$

## Data Simulation

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels import api
from scipy import stats
from scipy.optimize import minimize 

In [2]:
np.random.seed(123)

n_obs = 5000
n_x = 2
n_region = 2

# define parameters
b = np.array([[5,1,2], [10,5,8]])

# generate region id
id_region = []
for i in range(n_region):
    id_region += list(np.ones(n_obs, dtype=int)*i)

# generate x 
x = np.ones((n_obs*n_region, n_x+1))
x[:, 1:] = np.random.rand(n_obs*n_region, n_x) *10

# generate a normally distributed residual
e = np.random.normal(0, 1, n_obs*n_region)

# generate y
y = np.sum(x * b[id_region], axis=1)+e

In [3]:
# generate a dataframe
df = pd.DataFrame(x, columns=['const', 'x1', 'x2'])
df['y'], df['e'], df['id_region'] = y, e, id_region
df.iloc[np.append(np.arange(0, 5), np.arange(-5, 0)),:]

,const,x1,x2,y,e,id_region
0,1.0,6.964692,2.861393,17.851749,0.164271,0
1,1.0,2.268515,5.513148,17.487703,-0.807107,0
2,1.0,7.194690,4.231065,22.531296,1.874477,0
3,1.0,9.807642,6.848297,28.252163,-0.252074,0
4,1.0,4.809319,3.921175,18.523444,0.871775,0
9995,1.0,5.571459,1.589260,49.145193,-1.426185,1
9996,1.0,5.235259,7.247336,93.429420,-0.725566,1
9997,1.0,8.652763,7.978197,118.514427,1.425032,1
9998,1.0,1.271152,4.450733,52.188191,0.226565,1
9999,1.0,2.770733,2.799370,46.949035,0.700409,1


## OLS

In [4]:
# Pool OLS
ols_pool = api.OLS(df.y, df[['const', 'x1', 'x2']]).fit()
print(ols_pool.params)
print(f'srqt(SSR/n_obs) = {np.sqrt(ols_pool.ssr/len(df))}')

const    6.736266
x1       3.045522
x2       5.123648
dtype: float64
srqt(SSR/n_obs) = 29.407784795621332


In [5]:
# OLS by regions
for i in range(n_region):
    print(f'# ---- Region {i}: OLS parameters ----')
    features = df.loc[df.id_region==i, ['const', 'x1', 'x2']]
    ols_region = api.OLS(df.loc[df.id_region==i].y, features).fit()
    print(ols_region.params)
    print(f'srqt(SSR/n_obs) = {np.sqrt(ols_region.ssr/n_obs)}')

# ---- Region 0: OLS parameters ----
const    4.994363
x1       1.000662
x2       1.999587
dtype: float64
srqt(SSR/n_obs) = 1.0005487534245767
# ---- Region 1: OLS parameters ----
const    9.969552
x1       5.004759
x2       7.998240
dtype: float64
srqt(SSR/n_obs) = 0.9845582537636997


## MLE

Suppose we have $n$ observations and $p$ features (i.e., $p$ coefficients) and denote $X_i = (x_{i1}, \dots, x_{ip})^T$

$$f(y_i|\beta, \sigma^2) = N (X_i^T\beta, \sigma^2), \text{ where } \beta = (\beta_1, \dots, \beta_j, \dots, \beta_p)^T$$
$$ \beta_j \sim N(\mu_j, \eta_j^2), \text{ where } \mu = (\mu_1, \dots, \mu_p), \text{ and } \eta = (\eta_1, \dots, \eta_p)$$

Then the joint distribution is 
$$L(\beta, \sigma, \mu, \eta) = (\sqrt{2\pi}\sigma)^{-n}\exp\{\frac{\sum_{i=1}^n(y_i - X_i^T\beta)^2}{2\sigma^2}\} \times \prod_{j=1}^p (\sqrt{2\pi}\eta_j)^{-1} \exp\{\frac{(\beta_j - \mu_j)^2}{2\eta_j^2}\}$$

In [6]:
# Likelihood function for pool OLS
def ll_pool(par_init, X=df[['const', 'x1', 'x2']], y=df['y']):
    # data
    dt_x = X.copy()
    dt_y = y.copy()

    # par
    b0, b1, b2, sig = par_init

    # predict the output
    y_hat = dt_x @ np.array([b0, b1, b2])
    
    # Calculate the log-likelihood for normal distribution
    LL = stats.norm.logpdf(dt_y, y_hat, sig)
    
    # Calculate the negative log-likelihood
    neg_LL = -1*np.sum(LL)
    return neg_LL

In [7]:
# Summary MLE results
def mle_summary(mle_obj):
    print('MLE success: {0}\nMLE Obj func value: {1:.2f}\nMLE pars: {2}'.\
          format(mle_obj.success, mle_obj.fun, [round(x,2) for x in mle_obj.x]))

In [8]:
# minimize arguments: function, intial_guess_of_parameters, method
mle_pool = minimize(ll_pool, x0=np.array([1, 2, 2, 2]), method='L-BFGS-B')
mle_summary(mle_pool)

MLE success: True
MLE Obj func value: 48001.98
MLE pars: [6.74, 3.05, 5.12, 29.41]


In [9]:
# Add opt boundaries for x0
mle_pool = minimize(ll_pool, x0=np.array([1, 2, 2, 2]), method='L-BFGS-B', 
                    bounds=[(0,9), (0,9), (0,9), (1, 99)])
mle_summary(mle_pool)

MLE success: True
MLE Obj func value: 48001.98
MLE pars: [6.74, 3.05, 5.12, 29.41]


In [10]:
# Likelihood function for OLS by regions
def ll_region(par_init, X=df[['const', 'x1', 'x2']], y=df['y']):
    # data
    dt_x = X.copy()
    dt_y = y.copy()

    # par
    b0_0, b1_0, b2_0, sig_0, b0_1, b1_1, b2_1, sig_1 = par_init
    beta0 = np.array([b0_0, b1_0, b2_0])
    beta1 = np.array([b0_1, b1_1, b2_1])

    # predict the output
    y_hat_0 = dt_x[df.id_region==0] @ beta0
    y_hat_1 = dt_x[df.id_region==1] @ beta1

    # Calculate the log-likelihood for normal distribution
    LL_0 = stats.norm.logpdf(dt_y[df.id_region==0], y_hat_0, sig_0)
    LL_1 = stats.norm.logpdf(dt_y[df.id_region==1], y_hat_1, sig_1)
    
    # Calculate the negative log-likelihood
    neg_LL = -1*np.sum(np.append(LL_0, LL_1))
    return neg_LL

In [11]:
mle_region = minimize(ll_region, x0=np.array([1, 2, 3, 5, 1, 2, 3, 5]), method='L-BFGS-B')
mle_summary(mle_region)

MLE success: True
MLE Obj func value: 14114.32
MLE pars: [4.99, 1.0, 2.0, 1.0, 9.97, 5.0, 8.0, 0.98]


In [12]:
# Likelihood function for hierarchical OLS
def ll_hier(par_init, X=df[['const', 'x1', 'x2']], y=df['y']):
    # data
    dt_x = X.copy()
    dt_y = y.copy()

    # par
    dev0_0, dev1_0, dev2_0, sig_0, dev0_1, dev1_1, dev2_1, sig_1, \
    b0, sig_b0, b1, sig_b1, b2, sig_b2= par_init
    beta0 = np.array([b0+dev0_0*sig_b0, b1+dev1_0*sig_b1, b2+dev2_0*sig_b2])
    beta1 = np.array([b0+dev0_1*sig_b0, b1+dev1_1*sig_b1, b2+dev2_1*sig_b2])

    # predict the output
    y_hat_0 = dt_x[df.id_region==0] @ beta0
    y_hat_1 = dt_x[df.id_region==1] @ beta1

    # Calculate the log-likelihood for normal distribution
    LL_0 = stats.norm.logpdf(dt_y[df.id_region==0], y_hat_0, sig_0) + \
    stats.norm.logpdf(beta0[0], b0, sig_b0) + \
    stats.norm.logpdf(beta0[1], b1, sig_b1) + \
    stats.norm.logpdf(beta0[2], b2, sig_b2)
    LL_1 = stats.norm.logpdf(dt_y[df.id_region==1], y_hat_1, sig_1) + \
    stats.norm.logpdf(beta1[0], b0, sig_b0) + \
    stats.norm.logpdf(beta1[1], b1, sig_b1) + \
    stats.norm.logpdf(beta1[2], b2, sig_b2)
    
    # Calculate the negative log-likelihood
    neg_LL = -1*np.sum(np.append(LL_0, LL_1))
    return neg_LL

In [22]:
mle_hier = minimize(ll_hier, x0=np.array([.1,.2,.3,5,.1,.2,.3,5,1,1,2,1,3,1]), 
                    method='L-BFGS-B', 
                    bounds=[(-1,1), (-1,1), (-1,1), (0, 29), 
                            (-1,1), (-1,1), (-1,1), (0, 29),
                            (0,9), (.1,9), (0,9), (.1, 9), (0,9), (.1, 9)])
mle_summary(mle_hier)

MLE success: True
MLE Obj func value: 6490.96
MLE pars: [-0.0, -0.02, -0.02, 29.0, 0.0, 0.02, 0.02, 29.0, 6.74, 0.1, 3.05, 0.1, 5.12, 0.1]


In [24]:
mle_hier = minimize(ll_hier, x0=np.array([.1,.2,.3,5,.1,.2,.3,5,1,1,2,1,3,1]), 
                    method='L-BFGS-B', 
                    bounds=[(-9,9), (-9,9), (-9,9), (.1, 9), 
                            (-9,9), (-9,9), (-9,9), (.1, 9),
                            (0,9), (.1,9), (0,9), (.1, 9), (0,9), (.1, 9)])
mle_summary(mle_hier)

MLE success: True
MLE Obj func value: 33414.24
MLE pars: [-0.01, -0.09, -1.0, 1.02, 0.01, 0.09, 1.0, 9.0, 5.06, 0.1, 1.06, 0.1, 6.56, 4.6]
